In [1]:
pip install openai==1.54.4

  Obtaining dependency information for openai==1.54.4 from https://files.pythonhosted.org/packages/c7/d8/3e4cf8a5f544bef575d3502fedd81a15e317f591022de940647bdd0cc017/openai-1.54.4-py3-none-any.whl.metadata
  Using cached openai-1.54.4-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.54.4-py3-none-any.whl (389 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from PIL import Image
from openai import OpenAI
import base64
import openai
import pillow_heif
import pandas as pd
import os
import re

In [2]:
os.environ["OPENAI_API_KEY"] = "" # -> OpenAI API key 입력
client = OpenAI()

In [3]:
class Image2Text:
    # 생성자 함수
    def __init__(self, jpg_path, output_dir):
        self.jpg_path = jpg_path  # JPG 이미지 경로
        self.output_dir = output_dir  # JPG 파일을 저장할 디렉토리
        self.base_name = os.path.basename(self.jpg_path)  # 파일 경로에서 파일명만 추출

    # 이미지를 base64로 인코딩하여 반환
    def encode_image(self):
        with open(self.jpg_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 이미지에 대한 질문을 OpenAI API에 요청 후 응답 반환
    def ask_about_image(self, client):
        base64_image = self.encode_image()
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # -> gpt-4o-mini 모델 사용
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                '이 사진에 나와있는 음식은 무엇인가요? '
                                '음식 이름에는 따옴표("")를 붙여서 답변해 주세요. '
                                '예시: 이 사진에 나와 있는 음식은 "닭갈비"입니다.'
                            ),
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )
        return response.choices[0].message.content

    # 응답 내용에서 음식 이름 추출
    def extract_food_name(self, response_content):
        sentences = response_content.split('.')
        for sentence in sentences:
            if '"' in sentence:
                food_name = sentence.split('"')[1]
                return food_name
        return None

    # 월 추출
    def extract_month(self):
        month = self.base_name.split('-')[1]  # '2024-04-20.jpeg'에서 '04' 추출
        return int(month)  # 월을 정수로 반환

    # 계절 추출
    def get_season(self, month):
        if 3 <= month <= 5:
            return '봄'
        elif 6 <= month <= 8:
            return '여름'
        elif 9 <= month <= 11:
            return '가을'
        else:
            return '겨울'

    # 응답 내용과 음식 이름, 월, 계절을 데이터프레임에 저장하고 기록
    def save_to_dataframe(self, food_name, month, season, csv_path):
        data = {
            'food_name': [food_name],
            'month': [month],
            'season': [season]
        }
        df = pd.DataFrame(data)
        if not os.path.isfile(csv_path):
            df.to_csv(csv_path, index=False)
        else:
            df.to_csv(csv_path, mode='a', header=False, index=False)


output_dir = "./re_images_jpg"  # JPG 이미지 저장 경로
csv_path = "./food_names.csv"  # CSV 경로


In [4]:
# JPG 파일 경로를 리스트로 설정
jpg_paths = [
    "./KakaoTalk_Image_2024-11-27-00-15-19_008.jpeg",
    "./KakaoTalk_Image_2024-11-27-00-15-19_007.jpeg",
    "./KakaoTalk_Image_2024-11-27-00-15-19_003.jpeg",
    "./KakaoTalk_Image_2024-11-27-00-15-19_002.jpeg",
    "./KakaoTalk_Image_2024-11-27-00-15-19_001.jpeg"
]

# 각 JPG 이미지에 대해 처리
for jpg_path in jpg_paths:
    converter = Image2Text(jpg_path, output_dir)
    response_content = converter.ask_about_image(client)
    print(response_content)

    # 음식 이름 추출
    food_name = converter.extract_food_name(response_content)
    print(f"음식 이름: {food_name}")

    # 월과 계절 추출
    month = converter.extract_month()
    season = converter.get_season(month)
    print(f"월: {month}, 계절: {season}")

    # CSV 파일에 저장
    converter.save_to_dataframe(food_name, month, season, csv_path)

이 사진에 나와 있는 음식은 "해산물 파스타"입니다.
음식 이름: 해산물 파스타
월: 11, 계절: 가을
이 사진에 나와 있는 음식은 "피자"입니다.
음식 이름: 피자
월: 11, 계절: 가을
이 사진에 나와있는 음식은 "커리"입니다.
음식 이름: 커리
월: 11, 계절: 가을
이 사진에 나와 있는 음식은 "샐러드"입니다.
음식 이름: 샐러드
월: 11, 계절: 가을
이 사진에 나와 있는 음식은 "초밥"과 "육회"입니다.
음식 이름: 초밥
월: 11, 계절: 가을


In [5]:
df = pd.read_csv(csv_path)
df

,food_name,month,season
0,해산물 파스타,11,가을
1,피자,11,가을
2,커리,11,가을
3,샐러드,11,가을
4,초밥,11,가을
